# LangChain接入MCP

LangChain通过MultiServerMCPClient构建MCP服务端连接工具。

pip install langchain-mcp-adapters

In [3]:
pip install langchain-mcp-adapters



   ----------------- ---------------------- 3/7 [jsonschema-specifications]
   ---------------------- ----------------- 4/7 [jsonschema]
   ---------------------------- ----------- 5/7 [mcp]
   ---------------------------- ----------- 5/7 [mcp]
   ---------------------------- ----------- 5/7 [mcp]
   ---------------------------------------- 7/7 [langchain-mcp-adapters]



In [4]:
import asyncio
import json
import logging

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.chat_models import init_chat_model
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
class Configuration:
    def __init__(self) -> None:
        # 加载大模型的配置
        load_dotenv()
        self.api_key='你注册的deepseek api key'
        self.model="deepseek-chat"

    # 加载MCP配置
    @staticmethod
    def load_servers(file_path = "LangChain08.json"):
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f).get("mcpServers", {})

In [ ]:
async def run_chat_loop():
    """启动 MCP-Agent 聊天循环"""
    cfg = Configuration()
    servers_cfg = Configuration.load_servers()

    # 1️.连接多台 MCP 服务器 
    # (创建MCP Server连接对象)
    mcp_client = MultiServerMCPClient(servers_cfg)
    # (用.get_tools()方法将连接后的MCP Server中的方法全部转换为LangChain可用的工具函数, 形成一个列表)
    tools = await mcp_client.get_tools()  
    logging.info(f"✅ 已加载 {len(tools)} 个 MCP 工具： {[t.name for t in tools]}")

    # 2️.初始化大模型（DeepSeek / OpenAI / 任意兼容 OpenAI 协议的模型）
    llm = init_chat_model(
        model=cfg.model,
        model_provider="deepseek",
        api_key=cfg.api_key
    )

    # 3️. 构造 LangChain Agent（用通用 prompt）
    prompt = hub.pull("hwchase17/openai-tools-agent")
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    # 4️. CLI 聊天
    print("\n  MCP Agent 已启动，输入 'quit' 退出")
    while True:
        user_input = input("\n你: ").strip()
        if user_input.lower() == "quit":
            break
        try:
            result = await agent_executor.ainvoke({"input": user_input})
            print(f"\nAI: {result['output']}")
        except Exception as exc:
            print(f"\n⚠️  出错: {exc}")

In [ ]:
# 主程序入口
# if __name__ == "__main__":
#     logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
#     asyncio.run(run_chat_loop())

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
# asyncio.run()是用于运行协程函数（异步函数）
asyncio.run(run_chat_loop())
